# Keyword Extraction Analysis
Analyze buzzwords driving sentiment on any given day

In [461]:
import os
import pandas as pd
import numpy as np

### Download data from HF Hub

In [462]:
from huggingface_hub import HfApi

api = HfApi()
all_files = api.list_repo_files("hblim/top_reddit_posts_daily", repo_type="dataset")
parquet_files = sorted([f for f in all_files if f.startswith('data_scored') and f.endswith(".parquet")])

df = []
for shard in parquet_files:
    local_path = api.hf_hub_download(repo_id="hblim/top_reddit_posts_daily", filename=shard, repo_type="dataset")
    file_date = os.path.splitext(os.path.basename(local_path))[0]
    df.append(pd.read_parquet(local_path).assign(filedate=file_date))
df = pd.concat(df, ignore_index=True)
print(f"Total records across {df.filedate.nunique()} days: {len(df)}")

Total records across 16 days: 4520


In [464]:
df.head()

,subreddit,created_at,retrieved_at,type,text,score,post_id,parent_id,sentiment,confidence,filedate
0,apple,2025-04-14 11:19:50-05:00,2025-04-14 23:44:27.136181-05:00,post,iPhone 16e Helps Apple Take Q1 2025 Top Spot in Global Smartphone Market\n\n,655,1jz2xrw,None,1,0.9971,2025-04-14
1,apple,2025-04-14 11:00:16-05:00,2025-04-14 23:44:27.136181-05:00,comment,I've closed all rings every day starting on June 19 2015. This won't be a problem as long as I don't get run over or die.,9,mn2wpoi,t3_1jyzp05,1,0.9965,2025-04-14
2,apple,2025-04-14 11:59:56-05:00,2025-04-14 23:44:27.136181-05:00,post,"Smartphone tariffs are coming back in ‘a month or two,’ says Trump admin\n\n",194,1jz3wsi,None,0,0.9829,2025-04-14
3,apple,2025-04-14 11:59:56-05:00,2025-04-14 23:44:27.136181-05:00,comment,"This topic has been automatically locked due to being controversial and/or political by nature. However, the submission itself will remain accessible as long as it is related to Apple.\n\n\nThis decision was made by a bot based on specific keywords. If you feel that this was in error, please report it to the moderators so that it can be reviewed.\n \n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/apple) if you have any questions or concerns.*",1,mn38mac,t3_1jz3wsi,0,0.9972,2025-04-14
4,apple,2025-04-14 18:04:42-05:00,2025-04-14 23:44:27.136181-05:00,post,Apple to Analyze User Data on Devices to Bolster AI Technology\n\n,69,1jzcpwz,None,1,0.9976,2025-04-14


### Look at specific subreddit, date

In [562]:
# 1. Filter your dataframe
date = '2025-04-14'
subreddit = 'apple'
day_sub = (df['filedate'] == date) & (df['subreddit'] == subreddit) 

In [589]:
dftest = df[day_sub]
print("Daily aggregated sentiment stats")
print("Community Weighted Sentiment =",((2 * dftest['sentiment'] - 1) * np.log1p(dftest['score'].clip(0,None))).mean())
print("Average Sentiment =",dftest['sentiment'].mean())
# dftest.sort_values('score',ascending=False)
# dftest.groupby('parent_id').agg({'sentiment': ['mean','sum','count']})

Daily aggregated sentiment stats
Community Weighted Sentiment = 0.3353147742165998
Average Sentiment = 0.43902439024390244


### Use KeyBERT and sentiment transformers model to extract keywords

In [587]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import spacy

raw_text = " ".join(df.loc[day_sub, 'text'].astype(str))

# 2. Load spaCy with parser enabled for noun_chunks
nlp = spacy.load("en_core_web_sm")  # keep the parser on
doc = nlp(raw_text.lower())

# 3. Build candidate phrases
candidates = " ".join(
    [chunk.text for chunk in doc.noun_chunks]
    + [ent.text for ent in doc.ents if ent.label_ in {"PRODUCT","EVENT",}]
)

for exclude in ['google','pixel','android','apple','rationale','advice','blog','topic','locked','author','moderator','error','bot','comments','archive','support','discord']:
    candidates = candidates.replace(exclude,' ')

# 4. Keyword extraction with local embeddings
model    = SentenceTransformer("all-MiniLM-L6-v2")
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(
    candidates,
    keyphrase_ngram_range=(1, 3),
    stop_words="english",
    use_mmr=True,
    diversity=0.9,
    top_n=5
)

print(keywords)


[('smartphone market', 0.5024), ('command thanks universal', 0.0662), ('dimensions leak years', 0.0196), ('wwdc non paywall', -0.0052), ('animations new techniques', -0.0178)]


### Ensure keywords actually match to posts or comments based on cosine similarity

In [591]:
from sklearn.metrics.pairwise import cosine_similarity

# 1) Precompute embeddings for all texts in your day/subreddit slice
texts     = df.loc[day_sub, 'text'].tolist()
text_embs = model.encode(texts, convert_to_tensor=False)  # shape: (n_texts, 384)

results = []
subsets = {}
# if you only want to test on a single kw, iterate keywords_test instead
for kw, _score in keywords:  
    # kw is now a string
    kw_emb = model.encode(kw, convert_to_tensor=False)      # shape: (384,)
    kw_emb = kw_emb.reshape(1, -1)                          # shape: (1, 384)
    
    sims = cosine_similarity(text_embs, kw_emb).flatten()   # OK: (n_texts,) vs (1,384)
    
    # rank or threshold as before
    hits   = df.loc[day_sub].iloc[sims.argsort()[::-1]]
    mask   = sims >= 0.3
        
    subset = df.loc[day_sub].iloc[mask]
    if subset.empty:
        continue
    subsets[kw] = subset
    
    # compute sentiment stats on subset…
    mean_sent   = 2 * subset['sentiment'].mean() - 1
    weighted    = ((2 * subset['sentiment'] - 1) * np.log1p(subset['score'].clip(0,None))).mean()
    total_score = subset['score'].sum()
    results.append((kw, mean_sent, weighted, len(subset), total_score))

summary = pd.DataFrame(results, columns=[
    'keyword', 'mean_sentiment', 'community_weighted_sentiment', 'n_posts' , 'total_score'
]).sort_values('total_score', ascending=False).reset_index(drop=True)

summary

,keyword,mean_sentiment,community_weighted_sentiment,n_posts,total_score
0,smartphone market,-0.076923,0.841451,13,2798
1,dimensions leak years,-1.000000,-5.939423,2,804
2,animations new techniques,1.000000,2.944439,1,18
3,wwdc non paywall,-1.000000,-2.397895,1,10


### Manually inspect posts and comments associated with the keyword

In [593]:
keyword_index = 0
subsets[summary.keyword[keyword_index]].head().style.set_caption(f"KEYWORD = {summary.keyword[keyword_index]}")

,subreddit,created_at,retrieved_at,type,text,score,post_id,parent_id,sentiment,confidence,filedate
0,apple,2025-04-14 11:19:50-05:00,2025-04-14 23:44:27.136181-05:00,post,iPhone 16e Helps Apple Take Q1 2025 Top Spot in Global Smartphone Market,655,1jz2xrw,None,1,0.997100,2025-04-14
2,apple,2025-04-14 11:59:56-05:00,2025-04-14 23:44:27.136181-05:00,post,"Smartphone tariffs are coming back in ‘a month or two,’ says Trump admin",194,1jz3wsi,None,0,0.982900,2025-04-14
3,apple,2025-04-14 11:59:56-05:00,2025-04-14 23:44:27.136181-05:00,comment,"This topic has been automatically locked due to being controversial and/or political by nature. However, the submission itself will remain accessible as long as it is related to Apple. This decision was made by a bot based on specific keywords. If you feel that this was in error, please report it to the moderators so that it can be reviewed. *I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/apple) if you have any questions or concerns.*",1,mn38mac,t3_1jz3wsi,0,0.997200,2025-04-14
23,apple,2025-04-14 11:43:39-05:00,2025-04-14 23:44:27.136181-05:00,comment,My boss purchased me a 16e to use for iOS development. It may not make much sense for end users but it is a nearly perfect corporate phone.,309,mn35a6r,t3_1jz2xrw,1,0.998600,2025-04-14
24,apple,2025-04-14 11:24:30-05:00,2025-04-14 23:44:27.136181-05:00,comment,"Despite what the tech influencers might have said, Apple clearly knows what they’re doing.",283,mn31i1a,t3_1jz2xrw,1,0.999600,2025-04-14


### Helper tool: Retrieve post and comments by post_id

In [534]:
postid = '1k5ywd6'
df[lambda x: ((x.post_id == postid) | (x.parent_id == f't3_{postid}'))]

,subreddit,created_at,retrieved_at,type,text,score,post_id,parent_id,sentiment,confidence,filedate
2748,Android,2025-04-23 08:15:55-05:00,2025-04-23 19:03:18.888116-05:00,post,"The new feature that gives higher memory priority to background tabs containing user edits, such as fillable forms or drafts (reducing the chance of them being killed and thus not losing your progress) is now available in Chrome Canary for Android.\n\n",224,1k5ywd6,None,0,0.9717,2025-04-23
2749,Android,2025-04-23 08:43:37-05:00,2025-04-23 19:03:18.888116-05:00,comment,"Android's task refreshing is so bad and random that I've adapted my whole workflow around it by simply never trusting it and constantly copying whatever I input. If I write something and need switch away to another app even for a second, I copy the text before I do it. \n\nAndroid still does this even if you have 16GB of RAM!",1,molv84l,t3_1k5ywd6,0,0.9996,2025-04-23
2750,Android,2025-04-23 08:19:42-05:00,2025-04-23 19:03:18.888116-05:00,comment,"I love that ""it reduces the chance"" but it doesn't eliminate the chance something I am working on it is killed...",1,molr08u,t3_1k5ywd6,1,0.9835,2025-04-23
2751,Android,2025-04-23 08:17:05-05:00,2025-04-23 19:03:18.888116-05:00,comment,"Context: [**Background tabs containing user edits, such as filled forms or drafts, will soon have a higher memory priority in Chrome for Android, this will reduce the likelihood of these tabs been killed prematurely.**](https://old.reddit.com/r/Android/comments/1j3ktpg/background_tabs_containing_user_edits_such_as/)\n\n.\n\nThe patch responsible for this change [**was merged yesterday.**](https://chromium-review.googlesource.com/c/chromium/src/+/6321765)",1,molqjut,t3_1k5ywd6,0,0.9996,2025-04-23
2752,Android,2025-04-23 12:13:43-05:00,2025-04-23 19:03:18.888116-05:00,comment,"Would love it if Android would let me ""pin"" apps by default that I didn't want to come out of memory. Would be amazing for apps that are slow to re-open.",1,mon1t9v,t3_1k5ywd6,0,0.9928,2025-04-23
2753,Android,2025-04-23 11:59:23-05:00,2025-04-23 19:03:18.888116-05:00,comment,Solid upgrade for Android users.,1,momytb8,t3_1k5ywd6,1,0.9996,2025-04-23
